In [19]:
!pip install fastapi uvicorn mysql-connector-python --user


In [18]:
from fastapi import FastAPI, Request
import mysql.connector
import uuid
from datetime import datetime



In [17]:
app = FastAPI()

# Database configuration (replace with your actual database credentials)
db = mysql.connector.connect(
    host="191.101.13.199",
    user="u387197059_data_manager",
    password="0Tp/#bu2b",
    database="u387197059_aff_ana"
)
cursor = db.cursor()


In [10]:
import uuid

# Generate a unique session ID
session_id = str(uuid.uuid4())
ip_address = "192.168.0.1"  # Sample IP
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"  # Sample user agent

cursor.execute(
    "INSERT INTO sessions (session_id, ip_address, user_agent) VALUES (%s, %s, %s)",
    (session_id, ip_address, user_agent)
)
db.commit()
print("Session added with session_id:", session_id)


Session added with session_id: 0bda538e-1bcb-41c6-b8fa-2463aa1584f1


In [13]:
event_type = "button_click"
event_target = "contact_seller_button"
additional_data = '{"scroll_depth": "75%"}'

cursor.execute(
    "INSERT INTO events (session_id, event_type, event_target, additional_data) VALUES (%s, %s, %s, %s)",
    (session_id, event_type, event_target, additional_data)
)
db.commit()
print("Event logged for session_id:", session_id)


Event logged for session_id: 0bda538e-1bcb-41c6-b8fa-2463aa1584f1


In [ ]:
import requests
from fastapi import FastAPI, Request, HTTPException
from pydantic import BaseModel
from typing import Optional, Dict
import mysql.connector
import uuid
from datetime import datetime
from fastapi.middleware.cors import CORSMiddleware
import datetime
from typing import Optional
import json
import os
from fastapi.responses import JSONResponse  # ---- Tchoula: Import JSONResponse for custom response handling
from datetime import timedelta  # ---- Tchoula: Import timedelta for setting cookie expiration
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Database Connection in FastAPI
app = FastAPI()


# Database configuration using environment variables
db = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME")
)

'''db = mysql.connector.connect(
    host="191.101.13.199",
    user="u387197059_data_manager",
    password="0Tp/#bu2b",
    database="u387197059_aff_ana"
)'''
cursor = db.cursor()




# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["affordring.com"],  # Update this to the URL of your React frontend
    allow_credentials=True,
    allow_methods=["*"],  # Allows all HTTP methods (GET, POST, etc.)
    allow_headers=["*"],  # Allows all headers
)





#Start a New Session  (API Endpoint)
#This endpoint will initialize a new session and store details like IP address and user agent.



# Function to get city, region, and country from IP address
def get_location_from_ip(ip_address):
    try:
        response = requests.get(f"https://ipwho.is/{ip_address}")
        response.raise_for_status()  # Ensure a valid response
        location_data = response.json()

        city = location_data.get("city")
        region = location_data.get("region")
        country = location_data.get("country")
        return city, region, country
    except Exception as e:
        print("Error fetching location data:", e)
        return None, None, None

# Generate a unique session ID
class SessionResponse(BaseModel):
    session_id: str
    user_id: str
    city: Optional[str]  
    region: Optional[str]  
    country: Optional[str]  

@app.post("/start_session/", response_model=SessionResponse)
async def start_session(request: Request):
    # Generate a new user ID/etc
    user_id = str(uuid.uuid4())

    # Get user's IP address (for testing, replace with actual request IP in production)
    user_ip = request.client.host ##"176.168.212.40"  # Replace with `request.client.host` for real request IP
    city, region, country = get_location_from_ip(user_ip)  # Get city, region, and country

    # Check if the user_id exists in the users table
    cursor.execute("SELECT user_id FROM users WHERE user_id = %s", (user_id,))
    result = cursor.fetchone()

    # If the user_id does not exist, insert it into the users table
    if not result:
        cursor.execute(
            "INSERT INTO users (user_id, city, region, country, created_at) VALUES (%s, %s, %s, %s, %s)",
            (user_id, city, region, country, datetime.datetime.now())
        )

    # Generate a new session ID
    session_id = str(uuid.uuid4())

    # Insert the session data into the sessions table
    cursor.execute(
        "INSERT INTO sessions (session_id, user_id) VALUES (%s, %s)",
        (session_id, user_id)
    )
    db.commit()

    # ---- Tchoula: Create a custom response and set a cookie with an expiration time (e.g., 1 year)
    response = JSONResponse(content={"session_id": session_id, "user_id": user_id, "city": city, "region": region, "country": country})
    expiration_time = timedelta(days=365)  # ---- Tchoula: Set expiration to 1 year
    response.set_cookie(key="user_id", value=user_id, max_age=expiration_time.total_seconds(), httponly=True)  # ---- Tchoula: Set the user_id cookie with expiration

    return response  # ---- Tchoula: Return response with the set cookie




# Log an Event  (API Endpoint)
# This endpoint logs specific user events (like page views, button clicks) along with optional data like the target (e.g., product ID).

# Define a Pydantic model to represent the event data

# Define a Pydantic model to represent the event data
class LogEventModel(BaseModel):
    session_id: str
    event_type: str
    event_target: Optional[str] = None
    additional_data: Optional[Dict] = None

# Update the endpoint to accept a LogEventModel object
@app.post("/log_event/")
async def log_event(event: LogEventModel):
    # Convert additional_data to JSON string if provided
    additional_data = json.dumps(event.additional_data) if event.additional_data else "{}"

    # Insert data into the database
    try:
        cursor.execute(
            "INSERT INTO events (session_id, event_type, event_target, additional_data) VALUES (%s, %s, %s, %s)",
            (event.session_id, event.event_type, event.event_target, additional_data)
        )
        db.commit()
    except mysql.connector.Error as e:
        print(f"Error inserting data: {e}")
        return {"status": "error", "message": str(e)}

    return {"status": "success", "event_logged": event.event_type}





## Save search terms

class SearchTermRequest(BaseModel):
    term: str

# Path to the JSON file where search terms will be saved
FILE_PATH = "search_terms.json"

@app.post("/save_search")
async def save_search(search_request: SearchTermRequest):  # Updated to use the Pydantic model
    search_term = search_request.term

    # Print statement to check if the request is received
    print(f"Received search term: {search_term}")

    if not search_term:
        return {"error": "Search term is empty or invalid"}

    # Ensure the 'data' directory exists if using a custom path
    if not os.path.exists(os.path.dirname(FILE_PATH)) and os.path.dirname(FILE_PATH):
        os.makedirs(os.path.dirname(FILE_PATH))

    # Read existing data or create a new list
    if os.path.exists(FILE_PATH):
        with open(FILE_PATH, "r") as file:
            search_data = json.load(file)
    else:
        search_data = []

    # Append the new search term and save
    search_data.append(search_term)
    with open(FILE_PATH, "w") as file:
        json.dump(search_data, file, indent=4)

    return {"message": "Search term saved successfully"}


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8001)  # Use 80 for HTTP or 443 for HTTPS if SSL configured
